<a href="https://colab.research.google.com/github/g-roger/natural-language-process/blob/main/FilmeReviewsTrabalhoFinal_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

***Participantes (RM - NOME):***<br>
Gabriel Roger do Nascimento - 340399 <br>

###**Criar um classificador de sentimento aplicando técnicas de PLN**
---

Utilizando o dataset de revisões de filmes em português [1], criar um classificador de sentimentos que consiga um score na métrica F1 Score superior a 70%.

Devem utilizar uma amostra de 20% e randon_state igual a 42 para testar as implementações e mensurar a métrica F1 Score (usar o parâmetro average = 'weighted') o restante dos dados devem ser utilizados para o treinamento (80%).

Fique a vontade para testar os métodos de pré-processamento, abordagens, algoritmos e bibliotecas, mas explique e justifique suas decisões.
O trabalho poderá ser feito em grupo de até 4 pessoas (mesmo grupo do Startup One).

Separe a implementação do seu modelo campeão junto com a parte de validação/teste de forma que o professor consiga executar todo o pipeline do modelo campeão.

Composição da nota:
- 50% - Demonstrações das aplicações das técnicas de PLN (regras, pré-processamentos, tratamentos, variedade de modelos aplicados, etc.)
- 50% - Baseado na performance obtida com o dataset de teste (conforme recomendação da amostra) no seu modelo campeão e na validação que o professor processar (Métrica F1 Score).

[1] - https://dados-ml-pln.s3-sa-east-1.amazonaws.com/reviews-pt-br.csv

In [ ]:
# CARREGANDO O DATA FRAME
import pandas as pd
df = pd.read_csv('https://dados-ml-pln.s3-sa-east-1.amazonaws.com/reviews-pt-br.csv', encoding='utf-8')

# Façam o download do arquivo e utilizem localmente durante os testes

In [ ]:
df.info()

Bom desenvolvimento!

In [ ]:
df.head()

In [ ]:
!pip install spacy
!python -m spacy download pt

In [ ]:
import nltk
import spacy


from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier


Nós escolhemos SpaCy justamente por ter uma simplicidade e desempenho ótimos.

Uma grande vantagem é por ter modelos treinados para o português.

In [ ]:
nltk.download('stopwords')
nlp = spacy.load('pt')

In [ ]:
def lemmatizer(text):
  result = []
  for word in nlp(text):
      result.append(word.lemma_)
  return " ".join(result)

In [ ]:
def lemmatizer_verbs(text):
  sent = []
  doc = nlp(text)
  for word in doc:
      if word.pos_ == "VERB":
          sent.append(word.lemma_)
      else:
          sent.append(word.text)
  return " ".join(sent)

Estamos testando se mais efetivo utilizar a lemmatização de todo o texto ou apenas os verbos. A lemmatização é para deflexionar a palavra para seu lema.
Ex: gatos, gatas, gata são todas do mesmo lema gato.

In [ ]:
df['text_lemma'] = df.texto.apply(lemmatizer)

In [ ]:
df['text_lemma_verbs'] = df.texto.apply(lemmatizer_verbs)

In [ ]:
df.head()

In [ ]:
df_train, df_test = train_test_split(
    df,
    test_size=0.2,
    random_state=42
)

Unindo as stopwords das bibliotecas NLTK e Spacy para possuir uma quantidade que abrange mais palavras.

In [ ]:
stop_words = list(set(nlp.Defaults.stop_words).union(set(nltk.corpus.stopwords.words('portuguese'))))

Vetorização do documento
Estamos utilizando uma combinação de unigrama e bigrama com Gradient Boosting

O primeiro teste é com CountVectorizer e depois TfidfVectorizer


In [ ]:
#vectorizer = CountVectorizer(ngram_range=(1,1), stop_words=stop_words)
vectorizer = CountVectorizer(ngram_range=(1,2), stop_words=stop_words)
#vectorizer = CountVectorizer(ngram_range=(2,2), stop_words=stop_words)
vectorizer.fit(df_train.text_lemma_verbs)
vector = vectorizer.transform(df_train.text_lemma_verbs)

In [ ]:
# vectorizer = TfidfVectorizer(ngram_range=(1,2), stop_words=stop_words, use_idf=True, norm='l2')
# vectorizer.fit(df_train.text_lemma_verbs)
# vector = vectorizer.transform(df_train.text_lemma_verbs)

In [ ]:
gradient_boosting = GradientBoostingClassifier(random_state=42)

In [ ]:
gradient_boosting.fit(vector, df_train.sentimento)

In [ ]:
vector_test = vectorizer.transform(df_test.text_lemma_verbs)

In [ ]:
predict = gradient_boosting.predict(vector_test)

In [ ]:
f1_score(df_test.sentimento, predict, average="weighted")

TFIDF com RandomForest, unigrama e bigrama combinados

In [ ]:
#vectorizer2 = TfidfVectorizer(ngram_range=(1,2), stop_words=stop_words)
vectorizer2 = CountVectorizer(ngram_range=(1,2), stop_words=stop_words)
vectorizer2.fit(df_train.text_lemma_verbs)
vector2 = vectorizer.transform(df_train.text_lemma_verbs)

In [ ]:


forest = RandomForestClassifier(random_state=42)
forest.fit(vector2, df_train.sentimento)

vector_test2 = vectorizer2.transform(df_test.text_lemma_verbs)

predict2 = forest.predict(vector_test2)
f1_score(df_test.sentimento, predict2, average="weighted")

Unigrama e trigrama combinados com DecisionTree

In [ ]:
from sklearn.tree import DecisionTreeClassifier

vectorizer3 = CountVectorizer(ngram_range=(1,3), stop_words=stop_words)
vectorizer3.fit(df_train.text_lemma_verbs)
vector3 = vectorizer3.transform(df_train.text_lemma_verbs)

tree = DecisionTreeClassifier(random_state=42)

tree.fit(vector3, df_train.sentimento)

vector_test3 = vectorizer3.transform(df_test.text_lemma_verbs)



In [ ]:
predict3 = tree.predict(vector_test3) 
f1_score(df_test.sentimento, predict3, average="weighted")

####**Validação do professor**

Consolidar apenas os scripts do seu **modelo campeão**, desde o carregamento do dataframe, separação das amostras, tratamentos utilizados (funções, limpezas, etc.), criação dos objetos de vetorização dos textos e modelo treinado e outras 
implementações utilizadas no processo de desenvolvimento do modelo.



Os textos foram pré-processados com lemmatizer porém apenas para os verbos, 
para as palavras serem analisadas como um único item (lema da palavras).
Unimos as stop words de spacy e nltk.

Entre TfidfVectorizer e CountVectorizer, o CountVectorizer apresentou melhor resultado contendo uma contagem de tokens.

De todos os testes realizados o algoritmo que melhor trouxe resultado no F1-score foi RandomForest. 


In [ ]:
!pip install spacy
!python -m spacy download pt

import nltk
import spacy


from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier

import pandas as pd
df = pd.read_csv('https://dados-ml-pln.s3-sa-east-1.amazonaws.com/reviews-pt-br.csv', encoding='utf-8')

nltk.download('stopwords')
nlp = spacy.load('pt')

def lemmatizer_verbs(text):
  sent = []
  doc = nlp(text)
  for word in doc:
      if word.pos_ == "VERB":
          sent.append(word.lemma_)
      else:
          sent.append(word.text)
  return " ".join(sent)

df['text_lemma_verbs'] = df.texto.apply(lemmatizer_verbs)

df_train, df_test = train_test_split(
    df,
    test_size=0.2,
    random_state=42
)


stop_words = list(set(nlp.Defaults.stop_words).union(set(nltk.corpus.stopwords.words('portuguese'))))

vectorizer = CountVectorizer(ngram_range=(1,2), stop_words=stop_words)
vectorizer.fit(df_train.text_lemma_verbs)
vector = vectorizer.transform(df_train.text_lemma_verbs)

forest = RandomForestClassifier(random_state=42)
forest.fit(vector2, df_train.sentimento)

vector_test = vectorizer.transform(df_test.text_lemma_verbs)

predict = forest.predict(vector_test)
f1_score(df_test.sentimento, predict, average="weighted")
